In [1]:
import contextlib
import glob
import sys
import os

import pytorch_lightning as pl
import torch

from pathlib import Path
from nemo.core.config import hydra_runner

[NeMo W 2021-06-11 22:19:35 optimizers:46] Apex was not found. Using the lamb optimizer will error out.


In [2]:
SCRIPTS_DIR = os.path.realpath(Path(os.getcwd()) / '..' / 'src')

In [3]:
sys.path.append(SCRIPTS_DIR)

In [4]:
from models import EncDecCTCModelBPEAutoMin

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-06-11 22:19:38 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [5]:
audio_dir = '/home/dino/datasets/ICSI/Signals_WAV/Bro013'  # Path to a directory which contains audio files

# General configs
output_filename: str = "speech_to_text_transcriptions.txt"
amp: bool = False
audio_type: str = "wav"

cuda = torch.cuda.is_available()

In [6]:
asr_model = EncDecCTCModelBPEAutoMin.from_pretrained(
    model_name='stt_en_conformer_ctc_large', map_location='cuda:0'
)

[NeMo I 2021-06-11 22:19:38 cloud:56] Found existing object /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_large/45187441bcb7eebfac9a00c2adc9397e/stt_en_conformer_ctc_large.nemo.
[NeMo I 2021-06-11 22:19:38 cloud:62] Re-using file from: /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_large/45187441bcb7eebfac9a00c2adc9397e/stt_en_conformer_ctc_large.nemo
[NeMo I 2021-06-11 22:19:38 common:675] Instantiating model from pre-trained checkpoint


Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[NeMo I 2021-06-11 22:19:45 mixins:147] Tokenizer AutoTokenizer initialized with 128 tokens


[NeMo W 2021-06-11 22:19:45 modelPT:138] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/librispeech_withsp2/manifests/librispeech_train_withSP.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: false
    tarred_audio_filepaths: /data/asr_set_1.4/train_no_appen/train_no_appen__OP_0..1023_CL_.tar
    
[NeMo W 2021-06-11 22:19:45 modelPT:145] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/librispeech_withsp2/manifests/librivox-dev-other.json
    - /data/librispeech_withsp2/manifests/librivox

[NeMo I 2021-06-11 22:19:45 features:252] PADDING: 0
[NeMo I 2021-06-11 22:19:45 features:269] STFT using torch
[NeMo I 2021-06-11 22:19:48 modelPT:439] Model EncDecCTCModelBPEAutoMin was successfully restored from /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_large/45187441bcb7eebfac9a00c2adc9397e/stt_en_conformer_ctc_large.nemo.


In [7]:
trainer = pl.Trainer(gpus=int(cuda))
asr_model.set_trainer(trainer)
asr_model = asr_model.eval()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [8]:
# load paths to audio
filepaths = list(glob.glob(os.path.join(audio_dir, f"*.{audio_type}")))

@contextlib.contextmanager
def autocast():
    yield

In [9]:
torch.set_num_threads(1)

vad, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=True
)

Downloading: "https://github.com/snakers4/silero-vad/archive/master.zip" to /home/dino/.cache/torch/hub/master.zip


In [10]:
(get_speech_ts,
 get_speech_ts_adaptive,
 _, read_audio,
 _, _, _) = utils

In [11]:
transcriptions = {}
# transcribe audio
with autocast():
    with torch.no_grad():
        for i, filename in enumerate(filepaths):
            wav = read_audio(filename)
            speech_timestamps = get_speech_ts(
                wav, vad,
                num_steps=4
            )
            for i, timestamps in enumerate(speech_timestamps):
                offset = timestamps['start']
                onset = timestamps['end']
                filename_part = filename.replace('.wav', f'_{i}.wav')
                transcriptions[filename_part] = asr_model.transcribe(
                    [filename_part],
                    [wav[offset:onset]]
                )[0]

[NeMo W 2021-06-11 22:19:52 nemo_logging:349] /home/dino/tools/anaconda3/envs/automin/lib/python3.8/site-packages/torch/nn/modules/module.py:889: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:639.)
      result = self.forward(*input, **kwargs)
    
[NeMo W 2021-06-11 22:20:30 patch_utils:49] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.


In [12]:
transcriptions

{'/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_0.wav': 'unto his own exact',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_1.wav': 'rearedit p whose doors',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_2.wav': '',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_3.wav': 'it so',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_4.wav': '',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_5.wav': '',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_6.wav': '',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_7.wav': 'were talking about the meat',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_8.wav': 'proxes',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_9.wav': 'meeting recorder',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_10.wav': 'digits oh your no a cones iouteresting b goa the ass ry system and me and for wanting of',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/chan7_11.wav': 'he',
 '/home/dino/datasets/ICSI/Signals_WAV/Bro013/

In [ ]:
logging.info(f"Writing transcriptions into file: {output_filename}")
with open(output_filename, 'w', encoding='utf-8') as f:
    for line in transcriptions:
        f.write(f"{line}\n")